In [1]:
import json
import lightgbm as lgb
import numpy as np
import pandas as pd

from glob import glob
from tqdm import tqdm, tqdm_notebook

from utils import FEATS_EXCLUDED, COLS_TEST1, COLS_TEST2

pd.set_option('display.max_columns', 100)

%matplotlib inline

In [88]:
df = pd.read_csv('../input/calendar.csv')

In [89]:
# to datetime
df['date'] = pd.to_datetime(df['date'])
df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0


In [5]:
df.event_name_2.value_counts()

Father's day      2
Cinco De Mayo     1
OrthodoxEaster    1
Easter            1
Name: event_name_2, dtype: int64

In [135]:
df.event_name_1.value_counts()

NBAFinalsStart         6
SuperBowl              6
LentWeek2              6
Mother's day           6
PresidentsDay          6
Pesach End             6
LentStart              6
Pesach Start           6
Purim End              6
ValentinesDay          6
NBAFinalsEnd           6
Ramadan starts         6
StPatricksDay          6
MemorialDay            6
EidAlAdha              5
VeteransDay            5
IndependenceDay        5
Chanukah End           5
Christmas              5
NewYear                5
MartinLutherKingDay    5
Eid al-Fitr            5
Ramadan ends           5
Halloween              5
LaborDay               5
OrthodoxChristmas      5
ColumbusDay            5
OrthodoxEaster         5
Cinco De Mayo          5
Thanksgiving           5
Easter                 5
Father's day           4
Name: event_name_1, dtype: int64

In [111]:
# add ramadan end dates
ramadan_end_dates = ['2011-8-29','2012-8-18','2013-8-7','2014-7-27','2015-7-16','2016-7-5']
for d in ramadan_end_dates:
    df.loc[df['date']==d,'event_name_1'] = 'Ramadan ends'

In [117]:
# add Pesach start dates
pesach_start_dates = ['2011-4-18','2012-4-6','2013-3-25','2014-4-14','2015-4-3','2016-4-22']
for d in pesach_start_dates:
    df.loc[df['date']==d,'event_name_1'] = 'Pesach Start'

In [146]:
# add purim start dates
purim_start_dates = ['2011-3-19','2012-3-7','2013-2-23','2014-3-15','2015-3-4','2016-3-23']
for d in purim_start_dates:
    df.loc[df['date']==d,'event_name_1'] = 'Purim Start'

In [154]:
# add chanukah start dates
chanukah_start_dates = ['2011-12-21','2012-12-9','2013-11-28','2014-12-17','2015-12-7','2016-12-25']
for d in chanukah_start_dates:
    df.loc[df['date']==d,'event_name_1'] = 'Chanukah Start'

In [104]:
df['date'] = pd.to_datetime(df['date'])

In [155]:
is_nba_final = []
is_lent = []
is_ramadan = []
is_pesach = []
is_purim = []
is_chanukah = []

tmp_nba = 0
tmp_lent = 0
tmp_ramadan = 0
tmp_pesach = 0
tmp_purim = 0
tmp_chanukah = 0

for e in df['event_name_1']:
    if e == 'NBAFinalsStart':
        tmp_nba = 1
    is_nba_final.append(tmp_nba)
    if e == 'NBAFinalsEnd':
        tmp_nba = 0
        
    if e == 'LentStart':
        tmp_lent = 1
    is_lent.append(tmp_lent)
    if e == 'Easter':
        tmp_lent = 0

    if e == 'Ramadan starts':
        tmp_ramadan = 1
    is_ramadan.append(tmp_ramadan)
    if e == 'Ramadan ends':
        tmp_ramadan = 0

    if e == 'Pesach Start':
        tmp_pesach = 1
    is_pesach.append(tmp_pesach)
    if e == 'Pesach End':
        tmp_pesach = 0

    if e == 'Purim Start':
        tmp_purim = 1
    is_purim.append(tmp_purim)
    if e == 'Purim End':
        tmp_purim = 0
        
    if e == 'Chanukah Start':
        tmp_chanukah = 1
    is_chanukah.append(tmp_chanukah)
    if e == 'Chanukah End':
        tmp_chanukah = 0

In [156]:
df['is_NBA_final'] = is_nba_final
df['is_lent'] = is_lent
df['is_ramadan'] = is_ramadan
df['is_pesach'] = is_pesach
df['is_purim'] = is_purim
df['is_chanukah'] = is_chanukah

In [157]:
df[df['is_chanukah']==1]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_NBA_final,is_lent,is_ramadan,is_pesach,is_purim,is_chanukah
326,2011-12-21,11147,Wednesday,5,12,2011,d_327,Chanukah Start,NaN,NaN,NaN,0,0,0,0,1,0,0,0,1
327,2011-12-22,11147,Thursday,6,12,2011,d_328,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0,0,1
328,2011-12-23,11147,Friday,7,12,2011,d_329,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0,0,1
329,2011-12-24,11148,Saturday,1,12,2011,d_330,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0,0,1
330,2011-12-25,11148,Sunday,2,12,2011,d_331,Christmas,National,NaN,NaN,0,0,0,0,1,0,0,0,1
331,2011-12-26,11148,Monday,3,12,2011,d_332,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0,0,1
332,2011-12-27,11148,Tuesday,4,12,2011,d_333,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0,0,1
333,2011-12-28,11148,Wednesday,5,12,2011,d_334,Chanukah End,Religious,NaN,NaN,0,0,0,0,1,0,0,0,1
680,2012-12-09,11246,Sunday,2,12,2012,d_681,Chanukah Start,NaN,NaN,NaN,1,1,1,0,0,0,0,0,1
681,2012-12-10,11246,Monday,3,12,2012,d_682,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,1


In [150]:
df.is_lent.value_counts()

0    1384
1     585
Name: is_lent, dtype: int64

In [153]:
df[df.event_name_1=='Chanukah Start']

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_NBA_final,is_lent,is_ramadan,is_pesach,is_purim
326,2011-12-21,11147,Wednesday,5,12,2011,d_327,Chanukah Start,NaN,NaN,NaN,0,0,0,0,1,0,0,0
680,2012-12-09,11246,Sunday,2,12,2012,d_681,Chanukah Start,NaN,NaN,NaN,1,1,1,0,0,0,0,0
1034,2013-11-28,11344,Thursday,6,11,2013,d_1035,Chanukah Start,National,NaN,NaN,0,0,0,0,0,0,0,0
1418,2014-12-17,11446,Wednesday,5,12,2014,d_1419,Chanukah Start,NaN,NaN,NaN,0,0,0,0,0,0,0,0
1773,2015-12-07,11545,Monday,3,12,2015,d_1774,Chanukah Start,NaN,NaN,NaN,1,1,0,0,0,0,0,0


In [152]:
# add chanukah start dates
chanukah_start_dates = ['2011-12-21','2012-12-9','2013-11-28','2014-12-17','2015-12-7','2016-12-25']
for d in chanukah_start_dates:
    df.loc[df['date']==d,'event_name_1'] = 'Chanukah Start'